In [1]:
data_var = '2023-12-07'

In [2]:
import pandas as pd; pd.set_option('display.max_columns', None)
from datetime import date, timedelta
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [3]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [4]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [7]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
2453,2023-12-07,Israel Superliga,14:00,Hapoel Eilat,H. Haifa,1.55,2.23,171.5,1.85,1.81,-4.5,1.99,1.70,M32jymYu,0.645161,0.448430,0.540541,0.552486,0.093592,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,81,72,1.85,2.89,0.000,0.000,0.254409,0.015456,0.111144,0.00,0.000,inf,0.0,0.0,0.0,0.0,0.00,inf,0.0,0.0,0.0
2454,2023-12-07,Islândia Premier League,17:15,Grindavik,Stjarnan,2.02,1.72,176.5,1.87,1.79,1.5,1.87,1.83,8MB6SQnP,0.495050,0.581395,0.534759,0.558659,0.076445,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,83,84,3.37,1.54,0.000,0.000,0.113440,0.030912,0.015289,0.00,0.000,inf,0.0,0.0,0.0,0.0,0.00,inf,0.0,0.0,0.0
2455,2023-12-07,Islândia Premier League,16:15,Tindastoll,Hottur,1.35,3.02,165.5,1.83,1.87,-7.5,1.96,1.75,6cisMOPh,0.740741,0.331126,0.546448,0.534759,0.071867,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,79,85,1.70,2.13,0.000,0.000,0.540443,0.015289,0.080050,0.00,0.000,inf,0.0,0.0,0.0,0.0,0.00,inf,0.0,0.0,0.0
2456,2023-12-07,Islândia Premier League,16:15,Thorl,Hamar,1.12,5.72,186.5,1.86,1.84,-13.5,2.01,1.71,pr3YNpet,0.892857,0.174825,0.537634,0.543478,0.067682,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,96,85,2.05,4.22,0.000,0.000,0.951079,0.007644,0.114049,0.00,0.000,inf,0.0,0.0,0.0,0.0,0.00,inf,0.0,0.0,0.0
2457,2023-12-07,Islândia Premier League,16:15,Alftanes,Haukar,1.35,3.02,167.5,1.87,1.79,-7.5,2.01,1.72,ENfkK2f5,0.740741,0.331126,0.534759,0.558659,0.071867,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,90,93,2.37,1.66,0.000,0.000,0.540443,0.030912,0.109952,0.00,0.000,inf,0.0,0.0,0.0,0.0,0.00,inf,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2515,2023-12-07,China Wcba Feminina,08:30,Shanghai F,Zhejiang Golden Bulls F,1.55,2.34,159.5,1.88,1.79,-4.5,1.95,1.76,prwBx4r9,0.645161,0.427350,0.531915,0.558659,0.072512,151.976,75.055925,0.493867,2.4,1.341641,0.559017,276.34,1.710,0.998574,0.583961,113.0,298.774,253.679940,0.849070,2.4,1.341641,0.559017,95.79,3.716,3.441973,0.926258,56.0,82,93,3.37,1.03,0.000,0.000,0.287205,0.034681,0.072426,0.00,0.000,inf,0.0,0.0,0.0,0.0,0.00,inf,0.0,0.0,0.0
2516,2023-12-07,China Cba,08:35,Shandong,Jiangsu Dragons,1.11,6.21,195.5,1.85,1.89,-14.5,2.10,1.65,dIp48rwn,0.900901,0.161031,0.540541,0.529101,0.061931,288.054,170.290905,0.591177,0.6,1.341641,2.236068,428.76,2.924,1.632905,0.558449,-28.0,389.732,156.412175,0.401333,0.6,1.341641,2.236068,542.52,4.280,1.708025,0.399071,-51.0,108,99,3.97,5.48,272.258,0.000,0.985313,0.015125,0.169706,1.38,0.276,0.398551,0.0,0.0,0.0,0.0,0.00,inf,0.0,0.0,0.0
2517,2023-12-07,China Cba

## Apostas do dia

### Back Home V1 (QCut)

In [8]:
# Duplicando o dataset
df_bh_v1 = jogos.copy()

df_bh_v1 = df_bh_v1[(df_bh_v1['Odds_H'] >= 1.6) & (df_bh_v1['Odds_H'] <= 2.4)]

if not df_bh_v1.empty:    
    # Filtros pós modelo
    df_bh_v1 = df_bh_v1[((df_bh_v1['CG_A_Last_Game'] < 0.0) | (df_bh_v1['CG_A_Last_Game'] >= 160))]
    df_bh_v1 = df_bh_v1[((df_bh_v1['CV_HA'] < 0.0334) | (df_bh_v1['CV_HA'] >= 1.414))]
    df_bh_v1 = df_bh_v1[((df_bh_v1['P_Diff'] < 0.0464) | (df_bh_v1['P_Diff'] >= 0.065))]
    df_bh_v1 = df_bh_v1[((df_bh_v1['CG_A_Last_Game'] < 348) | (df_bh_v1['CG_A_Last_Game'] >= 920))]
    df_bh_v1 = df_bh_v1[((df_bh_v1['Goals_Last_H'] >= 79) & (df_bh_v1['Goals_Last_H'] <= 110)) | ((df_bh_v1['Goals_Last_H'] > 120.8) & (df_bh_v1['Goals_Last_H'] <= 143))]

    if not df_bh_v1.empty:
        df_bh_v1.to_csv(f'apostas_do_dia/{data_var}/back_home_v1_{data_var}.csv', index=False)

        print('------------ MÉTODO BACK HOME V1 (QCUT) ------------')
        print()

        df_bh_v1['Bet'] = 'Back Home'
        ligs = df_bh_v1['League'].unique().tolist()

        display(df_bh_v1[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

Sem jogos


### Back Home V2 (LR)

In [9]:
# Duplicando o dataset
df_bh_v2 = jogos.copy()

df_bh_v2 = df_bh_v2[(df_bh_v2['Odds_H'] >= 1.6) & (df_bh_v2['Odds_H'] <= 2.4)]
df_bh_v2.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v2.empty:
    mdl_bh_v2 = load_model('ML/back_home/v2/v2_back_home', verbose=False)
    prev = predict_model(mdl_bh_v2, df_bh_v2)
    prev = prev[prev['prediction_label'] == 1]

    # Filtros pós modelo
    prev = prev[((prev['CV_Odds_Geral_H'] <= 0.224) | (prev['CV_Odds_Geral_H'] > 0.441))]
    prev = prev[((prev['P(U)'] < 0.526) | (prev['P(U)'] > 0.535))]
    prev = prev[((prev['Last_Odd_H'] <= 1.59) | (prev['Last_Odd_H'] > 2.3))]

    
    if not df_bh_v2.empty:
        df_bh_v2.to_csv(f'apostas_do_dia/{data_var}/back_home_v2_{data_var}.csv', index=False)

        print('------------ MÉTODO BACK HOME V2 (LR) ------------')
        print()

        df_bh_v2['Bet'] = 'Back Home'
        ligs = df_bh_v2['League'].unique().tolist()

        display(df_bh_v2[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO BACK HOME V2 (LR) ------------



,Time,League,Home,Away,Odds_H,Bet
2454,17:15,Islândia Premier League,Grindavik,Stjarnan,2.02,Back Home
2465,14:00,Europa Liga Letônia,BK Liepaja,Pärnu,1.64,Back Home
2467,14:00,Libano Divisão 1,Antranik Beirut,Mayrouba,1.66,Back Home
2468,17:15,Portugal Lfb Feminina,CAB Madeira F,CDEFF F,1.93,Back Home
2469,13:00,Lituânia Nkl,Kretinga,Silute,1.63,Back Home
2473,23:00,México Lnbp,Astros,Fuerza Regia,1.90,Back Home
2477,16:45,Reino Unido Bbl,Manchester Giants,Surrey Scorchers,1.72,Back Home
2481,14:00,Rússia Liga Vtb United,Runa Basket Moscow,Uralmash Ekaterinburg,1.60,Back Home
2482,14:00,Europa Liga Aba,Cibona,Split,2.34,Back Home
2484,22:00,Eua Ncaa,North Dakota St.,Portland,1.80,Back Home


### Under V1 (QCut)

In [10]:
# Duplicando o dataset
df_u_v1 = jogos.copy()

df_u_v1 = df_u_v1[(df_u_v1['Odds_H'] >= 1.6) & (df_u_v1['Odds_H'] <= 2.4)]

if not df_u_v1.empty:    
    # Filtros pós modelo
    df_u_v1 = df_u_v1[((df_u_v1['CG_A_Last_Game'] <= 191) | (df_u_v1['CG_A_Last_Game'] > 280))]
    df_u_v1 = df_u_v1[((df_u_v1['Saldo_Gols_H'] <= 0) | (df_u_v1['Saldo_Gols_H'] > 16))]
    df_u_v1 = df_u_v1[((df_u_v1['CG_H_Last_Game'] <= 166.6) | (df_u_v1['CG_H_Last_Game'] > 260.85))]
    df_u_v1 = df_u_v1[((df_u_v1['Avg_CG_H'] <= 262.034) | (df_u_v1['Avg_CG_H'] > 969.8))]
    df_u_v1 = df_u_v1[((df_u_v1['MediaCustoGolAway'] <= 200.907) | (df_u_v1['MediaCustoGolAway'] > 868.35))]
    df_u_v1 = df_u_v1[((df_u_v1['P_Diff'] <= 0.0747) | (df_u_v1['P_Diff'] > 0.0971))]
    df_u_v1 = df_u_v1[((df_u_v1['HA_Odds_A'] >= 0) & (df_u_v1['HA_Odds_A'] <= 1.68)) | ((df_u_v1['HA_Odds_A'] > 1.8) & (df_u_v1['HA_Odds_A'] <= 1.9))]
    df_u_v1 = df_u_v1[((df_u_v1['CV_HA'] >= 0) & (df_u_v1['CV_HA'] <= 0.162))]
    df_u_v1 = df_u_v1[((df_u_v1['Last_Odd_A'] <= 1.297) | (df_u_v1['Last_Odd_A'] > 1.62))]
    df_u_v1 = df_u_v1[((df_u_v1['P(A)'] <= 0.234) | (df_u_v1['P(A)'] > 0.365))]
    
    if not df_u_v1.empty:
        df_u_v1.to_csv(f'apostas_do_dia/{data_var}/under_v1_{data_var}.csv', index=False)

        print('------------ MÉTODO UNDER V1 (QCUT) ------------')
        print()

        df_u_v1['Bet'] = 'Back Home'
        ligs = df_u_v1['League'].unique().tolist()

        display(df_u_v1[['Time', 'League', 'Home', 'Away', 'Odds_Under', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO UNDER V1 (QCUT) ------------



,Time,League,Home,Away,Odds_Under,Bet
2494,16:15,Europa Euroliga,Panathinaikos,Real Madrid,1.91,Back Home
